#### Author: Martín Gamboa

Analize the simulations for the study of the Frequency Point Spread Function (PSF). The instrument has angular and frequency resolution. Both are defined by the instrumental features.

The angular resolution can be approximated by the following expression: $FWHM_{\theta} = \frac{c}{\nu~(P-1)~\Delta h}$ \\

The frequency resolution is a bit more complex and in this notebook we study it with simulations. The set of simulations are at NERSC in the common directory within mmgamboa user.

For each simulation: 
* read files (two different formats)
* plot raw data
* fit gaussian 

In [ ]:
from importlib import reload
import pickle as pk
import os
import sys
import time
import glob

import healpy as hp
import numpy as np
import matplotlib as mp
import pandas as pd 

import scipy.stats as sc
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from scipy import interpolate

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.patches import Ellipse
from matplotlib.animation import FuncAnimation

from lmfit import Model
from astropy.io import fits
from scipy.interpolate import griddata
import lmfit 

import qubic
import qubic.fibtools as ft
from qubicpack.utilities import Qubic_DataDir

import AnalysisPSF as anpsf

#%matplotlib notebook

### Angular resolution

Fit gaussian to profiles.

I will fit the FWHM in the maps where the point source is cleary located in the band. For that I can plot manually and see the SNR level in the edge between band and take those maps where SNR are high and almost constant in the value

In [ ]:
def gaussian2d_b(x, amp, x0, y0, varx, vary):
    gauss = np.zeros((len(x),len(x)))
    #amp = 1 / (2 * np.pi * varx * vary)
    for i in range(len(x)):
        for j in range(len(x)):
            gauss[i,j] = amp * np.exp(- 0.5 * ( (x[i] - x0) ** 2 / varx ** 2 + \
                                                (x[j] - y0) ** 2 / vary ** 2)  )
            
    return gauss.ravel()

def gaussian2d_1sig(x, amp, x0, y0, var):
    gauss = np.zeros((len(x),len(x)))
    #amp = 1 / (2 * np.pi * varx * vary)
    for i in range(len(x)):
        for j in range(len(x)):
            gauss[i,j] = amp * np.exp(- 0.5 * ( ((x[i] - x0) ** 2 + \
                                                 (x[j] - y0) ** 2) / var ** 2) )
            
    return gauss.ravel()

def normalization(x,mapa):
    '''integral para normalizar la distribucion f(x,y)
    np.trapz(data,x,axis=[])'''    

    ef = np.trapz((np.trapz(mapa,x,axis=0)),x)
    return 1/ef

def f(val, fwhm):
    return np.nan_to_num(np.exp(- 0.5 * val ** 2 / (fwhm / np.sqrt(8 * np.log(2)) ) ** 2))


In [ ]:
d = qubic.qubicdict.qubicDict()
d.read_from_file('spectroimaging_article.dict')
nf_recon = d['nf_recon'][0]

In [ ]:
# Generate parameters to use. Up to now we reconstruct with only 4 sub-bands, so
nusfull = {}
nusedges = {}

_, nus_recon_edges, nus_recon, _,_,_ = qubic.compute_freq(d['filter_nu']/1e9, Nfreq = nf_recon, 
                   relative_bandwidth = d['filter_relative_bandwidth'])

nusfull.update({'4': nus_recon})
nusedges.update({'4': nus_recon_edges})

# Center
center = qubic.equ2gal(d['RA_center'], d['DEC_center'])

### Calibration

In [ ]:
# Measure parameters
NFREQ = 4
reso = 1.5
size = 200

fwhm = d['synthbeam_peak150_fwhm'] * 150 / nusfull["4"]

#Calibration parameters
nsidecal = 512
hpx = True
smooth = True
Ncalmaps = 10

nus_cal = np.linspace(150-150*.125,150+150*.125,Ncalmaps)
fwhm_cal = d['synthbeam_peak150_fwhm'] * 150 / nus_cal

xscale = np.linspace(- size / 2, size / 2, size) * reso / 60
xscalecal = np.copy(xscale)

pixel = hp.pixelfunc.ang2pix(nsidecal, np.deg2rad(90 - center[1]), 
                             np.deg2rad(center[0]), nest = False)


In [ ]:
def create_fake_maps_pointsources(nmaps, nside, fwhm, center_pix,
                                 xscale = None, hpx = True, smooth = True):
    # Create calibration maps with point source as flat spectrum
    g0_ud = np.zeros((nmaps, 12 * nside ** 2,))

    for j,fwhm_i in enumerate(fwhm):
        if hpx:
            if smooth:
                g0_ud[j, center_pix] = 1
                g0_ud[j, :] = 1e5*hp.smoothing(g0_ud[j, :], 
                                            fwhm = np.deg2rad(fwhm_i),
                                            verbose = False )
            else:
                vec_pix = hp.pix2vec(nside, center_pix)
                vec_pixeles = hp.pix2vec(nside, np.arange(12 * nside ** 2))
                ang_pixeles = np.arccos(np.dot(vec_pix,vec_pixeles) )
                g0_ud[j,:] = 1e5*f(ang_pixeles, fwhm = np.deg2rad(1.*fwhm_i) )
        else:
            g0_ud[j] = np.reshape(gaussian2d_b(xscale, 1e4, 0., 0., 
                                                   fwhm_i/np.sqrt(8*np.log(2)), fwhm_i/np.sqrt(8*np.log(2))),
                                       (len(xscale), len(xscale)))
    return g0_ud


In [ ]:
def fwhm_from_calibration(
    maps, x, nus_cal, th_fwhm, Ncalmaps, center, 
    reso = 1.5, size = 200, verbose = False):
    
    """maps.shape = (Ncalmaps, #pix, 3)"""
    
    delta_fwhm_g = np.zeros((Ncalmaps))
    fwhm_measured_g = []
    xscalecal = x
    
    gmodel_calb_g = Model(gaussian2d_b, independent_vars = ['x'])

    t0 = time.time()
    
    for j,fwhm_i in enumerate(th_fwhm):
        if verbose: print('Doing {}/{} sample'.format(j+1,Ncalmaps))
        sigma_i = fwhm_i / np.sqrt(8 * np.log(2))

        g0map = hp.gnomview(maps[j,:], rot = center, reso = reso,
                                   return_projected_map = True, xsize = size,
                                   no_plot = True)
        #
        if verbose:
            print("Maximum values of the map {:.2e}".format(np.max(g0_ud)))
            print("Initial FWHM = {:.2f}~deg".format(fwhm_i))

        g0map *= normalization(xscalecal, g0map)

        gmodel_calb_g.set_param_hint('amp', value = 1e4)
        gmodel_calb_g.set_param_hint('x0', value = 0)
        gmodel_calb_g.set_param_hint('y0', value = 0)
        gmodel_calb_g.set_param_hint('varx', value = sigma_i, )
        gmodel_calb_g.set_param_hint('vary', value = sigma_i, )

        params_calb_g = gmodel_calb_g.make_params()

        result_calb_g = gmodel_calb_g.fit(g0map.ravel(), params = params_calb_g, 
                                          x = xscalecal, y = xscalecal )

        if verbose:
            print('varx {:.2f} vary {:.2f}'.format(result_calb_g.best_values['varx'], 
                                               result_calb_g.best_values['vary']))

        fwhm_measured_i = np.sqrt(abs(result_calb_g.best_values['varx'])*abs(result_calb_g.best_values['vary'])) *\
                                np.sqrt(8*np.log(2))

        fwhm_measured_g.append(fwhm_measured_i)
        delta_fwhm_g[j] = fwhm_measured_i - fwhm_i



        if verbose: print('Delta(FWHM) {:.5f}'.format(delta_fwhm_g[j]))

    if verbose: print('Done in {:.2f} min'.format( (time.time() - t0) / 60 ) )

    fwhm_measured_g = np.array(fwhm_measured_g)
    parsfwhm_g = list(result_calb_g.best_values.values())
    
    d_calb = {'frequencies': nus_cal, 
              'fwhm_th': th_fwhm,
             'fwhm_mes': fwhm_measured_g,
             'delta_fwhm': delta_fwhm_g}
    
    df_calb = pd.DataFrame(data = d_calb)
    
    return df_calb

In [ ]:
g0_ud = create_fake_maps_pointsources(Ncalmaps, nsidecal, fwhm_cal, pixel)

In [ ]:
cal_df = fwhm_from_calibration(g0_ud, xscalecal, nus_cal, fwhm_cal, Ncalmaps, center)

#### Reading new simulation of point source

In [ ]:
#fileloc_0 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg8000_0.fits'
#fileloc_1 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg8000_1.fits'

##tol=1e-4 nrec = 4
#fileloc_2 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg4000_3.fits'

##tol = 1e-5 nrec = 4
#fileloc_3 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg4000-tol1e-05_4.fits'
#fileloc_4 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg4000-tol1e-05_5.fits'

## tol = 1e-5 nrec = 4 nside = 512
fileloc_4 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg4000-tol1e-05_6.fits'
#fileloc_5 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg3000-tol1e-05_8.fits'
#fileloc_5 = 'angularresolution/new-PS_sim_nfsub15-nfrec5-pntg3000-tol1e-05_14.fits'

## tol = 1e-5 nrec = 4 nside = 1024
#fileloc_4 = 'angularresolution/new-PS_sim_nfsub16-nfrec4-pntg3000-tol1e-05_7.fits'

simu_1 = fits.open(fileloc_4)
#simu_2 = fits.open(fileloc_5)

maps_recon_1 = simu_1['MAPS_RECON'].data
maps_convo_1 = simu_1['MAPS_CONVOLVED'].data
#maps_recon_2 = simu_2['MAPS_RECON'].data
#maps_convo_2 = simu_2['MAPS_CONVOLVED'].data

### Compute FWHM in simulated maps! 

In [ ]:
def fwhm_from_maps(d, maps, nus, x, center, 
                   reso = 1.5, size = 200, verbose = False):

    t1 = time.time()
    NFREQ = len(maps)
    arrMapsRec = np.array([maps,])
    fwhmMeasured = np.zeros((len(arrMapsRec), NFREQ)) 
    
    result_fits = []
    varx, vary = [], []
    for j_c, maps_recon_i in enumerate(arrMapsRec):
        for j, imap in enumerate(maps_recon_i):
            s0map = hp.gnomview(imap.T[0] / 1e5, rot = center, reso = reso,
                                           return_projected_map = True, xsize = size,
                                           no_plot = True)

            gmodel = Model(gaussian2d_b, independent_vars=['x',], )
            #Set initial guess 
            fwhm_s0init = d['synthbeam_peak150_fwhm'] * 150 / nus[j]

            params = gmodel.make_params(amp = 1, x0 = 0, y0 = 0, 
                                        varx = fwhm_s0init / np.sqrt(8 * np.log(2)), 
                                        vary = fwhm_s0init / np.sqrt(8 * np.log(2)))

            result_s0 = gmodel.fit(s0map.ravel(), params = params, 
                                   x = x, y = x)

            fwhm_s0 = np.sqrt(result_s0.best_values['varx'] * result_s0.best_values['vary']) *\
                                                                                    np.sqrt(8 * np.log(2))

            fwhmMeasured[j_c, j] = fwhm_s0
            
            result_fits.append(result_s0)
            varx.append(result_s0.best_values['varx'])
            vary.append(result_s0.best_values['vary'])
            if verbose: print('Done {}/{}, fwhm = {}'.format(j + 1, len(arrMapsRec[0]), fwhm_s0))
    if verbose: print('All done in {:.4f} seconds'.format((time.time() - t1) ) )
        
    diff_fwhm = np.squeeze(np.array(fwhmMeasured - d['synthbeam_peak150_fwhm'] * 150 / nus))
    d_data = {'frequencies': nus, 
              'fwhm_th': np.array(d['synthbeam_peak150_fwhm'] * 150 / nus),
             'fwhm_mes': fwhmMeasured[0],
             'delta_fwhm': diff_fwhm,
             'varx': np.array(varx),
             'vary': np.array(vary)}
    
    df_data = pd.DataFrame(data = d_data)
    
    return df_data, result_fits

In [ ]:
data_df, result_fit = fwhm_from_maps(d, maps_recon_1, nusfull['4'], xscale, center)
data_sim, result_sim = fwhm_from_maps(d, maps_convo_1, nusfull['4'], xscale, center)

Look at the maps in one FITing

In [ ]:
jj = 0
s0map = hp.gnomview(maps_recon_1[jj,:,0] / 1e5, rot = center, reso = reso,
                                           return_projected_map = True, xsize = size,
                                           no_plot = True)
X, Y = np.meshgrid(xscale, xscale)
fig, axs = plt.subplots(1, 3, figsize=(10, 3))

vmax = np.nanpercentile(s0map, 99.9)
vmin = -0.5
ax = axs[0]
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
art = ax.pcolor(X, Y, s0map, vmin=vmin, vmax=vmax)#, shading='auto')
plt.colorbar(art, ax=ax, label='z')
ax.set_title('Map')

ax = axs[1]
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
fit = np.reshape(gaussian2d_b(xscale, **result_fit[jj].best_values), (len(xscale), len(xscale)))
art = ax.pcolor(X, Y, fit, vmin=vmin, vmax=vmax)#, shading='auto')
plt.colorbar(art, ax=ax, label='z')
ax.set_title('Fit')

ax = axs[2]
#fit = model.func(X, Y, **result_s0.best_values)
ax.set_xlim(-1,1)
ax.set_ylim(-1,1)
art = ax.pcolor(X, Y, s0map - fit, vmin=0.3*vmin, vmax=0.3*vmax)#, shading='auto')
plt.colorbar(art, ax=ax, label='z')
ax.set_title('Diff')
fig.tight_layout()
#plt.savefig("/home/martin/Doctorado/Tesis-Doctoral/Figuras_Tesis_Overleaf/fwhm_sketch")

proj = hp.projector.CartesianProj(rot=center, coord=None, 
                                  xsize=size, ysize=size, 
                                  lonra=None, latra=None)
long,lat = lambda th, ph: np.array([th,ph]) + np.array([360,0])
hp.gnomview(maps_recon_1[jj,:,0] / 1e5, rot = center, max = 1)
hp.get_all_neighbours(nside, hp.pix2ang(nside, ))
#maps_recon_1[jj,:,0] / 1e5
#hp.boundaries()

### Fit the calibration difference 
and then removes the interpolated value from data maps

In [ ]:
def model_fwhm_diff(x,a,b):
    return a * x + b

xSPSF = d['synthbeam_peak150_fwhm'] * 150 / fwhm_cal
ySPSF_diff = cal_df["delta_fwhm"]#delta_fwhm_g

# Model 2 or g
gmodel_fwhm_diff = Model(model_fwhm_diff, independent_vars=['x',], )
params_fwhm_diff = gmodel_fwhm_diff.make_params(a = 1, b = 1)
result_fwhm_diff = gmodel_fwhm_diff.fit(ySPSF_diff, params_fwhm_diff, x = xSPSF)
parsfwhm_diff = list(result_fwhm_diff.best_values.values())

In [ ]:
cal_df.keys()

In [ ]:
#plt.plot(cal_df['frequencies'], cal_df["delta_fwhm"], 'k-')
plt.plot(cal_df['frequencies'], cal_df["fwhm_th"], 'ro-', alpha= 0.5)
plt.plot(cal_df['frequencies'], cal_df["fwhm_mes"], 'ro-')
plt.quiver([cal_df['frequencies'][0]],
           [cal_df["fwhm_mes"][0]],
           [0],
          [cal_df["fwhm_th"][0] - cal_df["fwhm_mes"][0] ])
#plt.axhline(hp.nside2resol(512, arcmin = True) / 60, ls = "--", c = "k")

In [ ]:
import matplotlib.patches as patches


In [ ]:
fig,ax = plt.subplots(nrows = 1, ncols = 1, figsize = (8, 5))

plt.rc('font', size = 14)

fwhmTh = lambda nu: d['synthbeam_peak150_fwhm'] * 150 / nu

ax.set_xlabel(r'$\nu~$[GHz]', fontsize = 14)
ax.set_ylabel(r'FWHM$~$[deg]', fontsize = 14)

ax.plot(nusfull["4"], 
        data_df["fwhm_mes"],
        color = 'r', marker = '*', ls = "", ms = 10,
        alpha = 0.5,
        label = 'FWHM measured')

#ax.plot(nusfull["4"], 
#        data_df["fwhm_mes"] - model_fwhm_diff(nusfull["4"], 
#                                              *parsfwhm_diff) ,
#        color = 'r', marker = '*', ls = "", ms = 10,
#        label = 'FWHM correct')
#Plot with error bars --> \pm0.05. Uncertainity comes from plane-healpix projection 
ax.errorbar(nusfull["4"], 
        data_df["fwhm_mes"] - model_fwhm_diff(nusfull["4"], 
                                              *parsfwhm_diff) ,
        yerr = 0.01/2,
        color = 'r', marker = '*', ls = "", ms = 10,
        label = 'FWHM correct')

# Plot theoretical line
plt.plot(cal_df['frequencies'], cal_df["fwhm_th"], 'b-', alpha= 0.5)

ax.plot(nusfull["4"], fwhmTh(nusfull["4"]),#  yerr = np.diff(fwhmTh(nusedges['4']))/2,
        color = 'b', marker = 'o', ms = 10, alpha = 0.5, ls = "",# capsize = 5,
        label = 'FWHM theoretical')
for i in range(4):
    ax.add_patch(
     patches.Rectangle(
        (nusedges['4'][i], fwhmTh(nusedges['4'][i+1])),
        np.diff(nusedges['4'])[i], #ancho
        fwhmTh(nusedges['4'][i]) - fwhmTh(nusedges['4'][i+1]), #altura
        edgecolor = 'blue',
        facecolor = 'blue',
        fill=True, alpha = 0.1
     ) )

ax.set_xlim(nusedges['4'][0]*0.99, nusedges['4'][-1]*1.01)
ax.legend(fontsize = 14)
ax.tick_params(axis = 'both', bottom = True, top = True, 
               left = True, right = True, direction = 'in')
ax.grid()
#
plt.tight_layout() #makes subplots nicely fit in the figure.
#plt.savefig('Angular-Resolution_reso{}_nside{}_rectangles.pdf'.format(reso, nsidecal), format = 'pdf')
#plt.savefig('/home/mgamboa/Doctorado/Tesis-Doctoral/Figuras_Tesis_Overleaf/Angular-Resolution_reso{}_nside{}_rectangles.pdf'.format(reso, nsidecal), format = 'pdf')

In [ ]:
print("Elipticidad")
#(data_df['varx'] - data_df['vary']) / (data_df['varx'] + data_df['vary']) 
print(1 - data_df['vary'] / data_df['varx'])
print(1 - data_sim['vary'] / data_sim['varx'])
print((1 - data_df['vary'] / data_df['varx'])/(1 - data_sim['vary'] / data_sim['varx']))

In [ ]:
qreso = lambda nu : d['synthbeam_peak150_fwhm'] * 150 / nu * 60
print("nside \t hpreso r@130 \t r@140 \t r@150 \t r@160 \t r@170 \t r@190 \t r@210 \t r@230 \t r@250 " )
for i in [8,16,32,64,128,256,512]:
    resol = hp.nside2resol(i, arcmin = True)
    print("{} \t {:5.1f} \t {:.2f} \t {:.2f} \t {:.2f} \t {:.2f} \t {:.2f}\
     {:.2f} \t {:.2f} \t {:.2f} \t {:.2f}".format(i, resol, 
                                          qreso(130)/resol, qreso(140)/resol,
                                          qreso(150)/resol, qreso(160)/resol,
                                          qreso(170)/resol, qreso(190)/resol, 
                                                  qreso(210)/resol, qreso(230)/resol, 
                                                  qreso(250)/resol,))

quick computation of time for each simulation. Data taken from iris

In [ ]:
sisi = np.array([1653,3343,2770,5085,8652])
sisi2=np.array([1604,3507,2723,4976,9036])
sisi3=np.array([1524,3384,2860,4575,8518])
sisi4=np.array([1536,3766,2737,5415,8993])
nsisi= sisi/np.sum(sisi)
nsisi2= sisi2/np.sum(sisi2)
nsisi3= sisi3/np.sum(sisi3)
nsisi4= sisi4/np.sum(sisi4)

t0,t2,t3,t4 = 2571.95*nsisi/60, 2794.24*nsisi2/60, 2495.46*nsisi3/60,2666.2*nsisi4/60
print(t0[1:-1])
print('average recon 2,3,4 sub-freqs {:.2f}hs'.format(np.sum(t0[1:-1])), 2571/60)
print('{:.2f}hs'.format(np.sum(t2[1:-1])), 2794/60)
print('{:.2f}hs'.format(np.sum(t3[1:-1])), 2495/60)
print('{:.2f}hs'.format(np.sum(t4[1:-1])), 2666/60)


Pedagogical plot

In [ ]:
data_3bands['band0'][0],data_3bands['band1'][0],data_3bands['band2'][0]

In [ ]:

#for ibnd in [2,3,4,5]:
ibnd = 3
fig = plt.figure(figsize=(4, 4))
ax = fig.gca()
data_array = locals()['data_{}bands'.format(ibnd)]

#Generate xvals to plot spline
xfit = data_dict['Data_fit'][0]
xvals = np.linspace(np.min(xfit), np.max(xfit), 1000)

nsimu = len(data_array['nu_in'])
points, = ax.plot(data_array['nu_rec'], 
                  np.array([data_array['band{}'.format(i)] for i in range(ibnd)]).T[0], 'ko', alpha = 0.3)
line = ax.axvline(data_array['nu_in'][0], color = 'k', ls = '--', alpha = 0.3)

#line_spl, = ax.plot(xvals+data_array['nu_in'][0],
#                    data_array['SplFunc'](xvals), color = 'k', ls = '-')
#print(line_spl)
text = ax.text(140, 1, r'$\nu_i=${:3.2f} GHz'.format(data_array['nu_in'][0]), fontsize=14)

def init():
    ax.axhline(0, color = 'k', ls = '-', alpha = 0.2)
    c = ['b', 'g', 'y', 'orange', 'r']
    for ib in range(ibnd):
        ax.axvspan(xmin = data_array['nu_edge'][ib], xmax = data_array['nu_edge'][ib + 1], 
                   alpha = 0.2, color = c[ib])
    ax.set_xlim(130,170)
    ax.set_xlabel(r'$\nu$ [GHz]')
    ax.set_ylabel(r'$I_{out}$')
    ax.set_ylim(-0.2,1.1)
    return points

def animate(i):
    #points.set_data(data_array['nu_rec'], 
    #                np.array([data_array['band{}'.format(i)] for i in range(ibnd)]).T[i])
    ax.plot(data_array['nu_rec'], 
                    np.array([data_array['band{}'.format(i)] for i in range(ibnd)]).T[i], 'ko', alpha = 0.3)
    line.set_xdata(data_array['nu_in'][i])
    #line_spl.set_xdata(xvals + data_array['nu_in'][i])
    text.set_text(r'$\nu_i=${:3.2f} GHz'.format(data_array['nu_in'][i]))
    return(points)


steps = np.arange(nsimu)
anim = FuncAnimation(fig, animate, steps, init_func=init, interval=1000, blit=False, repeat=False)

# Save a .gif
anim.save('./{}FPSF_pedagogical.gif'.format(ibnd), writer='imagemagick')

In [ ]:
#for ibnd in [2,3,4,5]:
ibnd = 3
fig = plt.figure(figsize=(4, 4))
ax = fig.gca()
data_array = locals()['data_{}bands'.format(ibnd)]

#Generate xvals to plot spline
xfit = data_dict['Data_fit'][0]
xvals = np.linspace(np.min(xfit), np.max(xfit), 1000)

nsimu = len(data_array['nu_in'])
points, = ax.plot(data_array['nu_rec'], 
                  np.array([data_array['band{}'.format(i)] for i in range(ibnd)]).T[0], 'ko', alpha = 0.3)
line = ax.axvline(data_array['nu_in'][0], color = 'k', ls = '--', alpha = 0.3)
text = ax.text(140, 1, r'$\nu_i=${:3.2f} GHz'.format(data_array['nu_in'][0]), fontsize=14)

def init():
    ax.axhline(0, color = 'k', ls = '-', alpha = 0.2)
    c = ['b', 'g', 'y', 'orange', 'r']
    for ib in range(ibnd):
        ax.axvspan(xmin = data_array['nu_edge'][ib], xmax = data_array['nu_edge'][ib + 1], 
                   alpha = 0.2, color = c[ib])
    ax.set_xlim(130,170)
    ax.set_xlabel(r'$\nu$ [GHz]')
    ax.set_ylabel(r'$I_{out}$')
    ax.set_ylim(-0.2,1.1)
    return points

def animate(i):
    ax.plot(data_array['nu_rec'], 
            np.array([data_array['band{}'.format(i)] for i in range(ibnd)]).T[0] - ,
            'ko', alpha = 0.3)
    line.set_xdata(data_array['nu_in'][i])
    #line_spl.set_xdata(xvals + data_array['nu_in'][i])
    text.set_text(r'$\nu_i=${:3.2f} GHz'.format(data_array['nu_in'][i]))
    return(points)


steps = np.arange(nsimu)
anim = FuncAnimation(fig, animate, steps, init_func=init, interval=1000, blit=False, repeat=False)

# Save a .gif
anim.save('./{}FPSF_pedagogical.gif'.format(ibnd), writer='imagemagick')